In [1]:
from keras.models import load_model
import pandas as pd
import pickle
import tensorflow as tf
import numpy as np
import keras
import matplotlib.pyplot as plt
import shap
import seaborn as sns
import os
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_curve, accuracy_score
from scipy import interpolate

2022-12-06 11:08:28.146333: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-06 11:08:29.262036: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /sw/external/libraries/cudnn-linux-x86_64-8.4.1.50_cuda11.6-archive/lib:/sw/spack/delta-2022-03/apps/cuda/11.6.1-gcc-11.2.0-vglutoe/extras/CUPTI/lib64:/sw/spack/delta-2022-03/apps/cuda/11.6.1-gcc-11.2.0-vglutoe/lib64:/sw/spack/delta-2022-03/apps/openmpi/4.1.2-gcc-11.2.0-37px7gc/lib:/sw/spack/delta-2022-03/apps/ucx/1.11.2-gcc-11.2.0-pymppfm/lib:/sw/spack/delta-2022-03/apps/gcc/11.2.0-gcc-8.4.1-fxgns

In [2]:
## List of models
all_models = [f for f in os.listdir("../../models/TopoDNN/trained_models/")]
print("\n".join(all_models))

topodnnmodel_pt
topodnnmodel_pt0
topodnnmodel_v3
topodnnmodel_standardize_pt
topodnnmodel_30
topodnnmodel_v1
topodnnmodel_v2
topodnnmodel_v4
topodnnmodel


In [3]:
def eval(model, mode, df_test):
    if mode == '_pt0':
        df_test = df_test.loc[:, df_test.columns != 'pt_0']
    elif mode == '_pt':
        pt_cols = [col for col in df_test.columns if 'pt' in col]
        df_test = df_test.drop(pt_cols, axis=1)
    elif mode == '_30':
        df_test = df_test.iloc[:,:30]
    elif mode == '_standardize_pt':
        pt_cols = [col for col in df_test.columns if 'pt' in col]
        df_test[pt_cols] = (df_test[pt_cols] - df_test[pt_cols].mean())/df_test[pt_cols].std()
    feats = np.array(df_test)
    data = np.copy(feats)
    return model(data)

In [8]:
for model_name in all_models:
    #Loading data
    df_test = pd.read_pickle("../../datasets/topoprocessed/test.pkl")
    labels = np.array(df_test.loc[:,df_test.columns == 'is_signal_new']).reshape(-1)
    df_test = df_test.loc[:,df_test.columns != 'is_signal_new']
    mode = model_name.replace('topodnnmodel', '')
    #Load model
    model = load_model("../../models/TopoDNN/trained_models/"+model_name)
    nparams = np.sum([np.prod(v.get_shape()) for v in model.trainable_weights])
    results = np.array(eval(model, mode, df_test)).flatten()
    
    auc = roc_auc_score(labels, results)*100
    fpr, tpr, thresholds = roc_curve(labels, results)
    intrp = interpolate.interp1d(tpr, fpr)
    brr = 1./intrp(0.3)
    
    results[results >= 0.5] = 1
    results[results < 0.5] = 0
    acc = accuracy_score(labels, results)*100
    print("{} \t\t Params: {}\t ROC-AUC: {:.2f}%, Accuracy: {:.2f}%, BRR: {:.2f}".format(model_name,nparams,auc,acc,brr))

topodnnmodel_pt 		 Params: 50323	 ROC-AUC: 95.84%, Accuracy: 89.67%, BRR: 125.88
topodnnmodel_pt0 		 Params: 59023	 ROC-AUC: 97.01%, Accuracy: 91.42%, BRR: 266.73
topodnnmodel_v3 		 Params: 27737	 ROC-AUC: 97.12%, Accuracy: 91.55%, BRR: 267.44
topodnnmodel_standardize_pt 		 Params: 59323	 ROC-AUC: 97.10%, Accuracy: 91.51%, BRR: 298.69
topodnnmodel_30 		 Params: 41323	 ROC-AUC: 96.23%, Accuracy: 89.47%, BRR: 278.12
topodnnmodel_v1 		 Params: 42003	 ROC-AUC: 97.15%, Accuracy: 91.56%, BRR: 309.21
topodnnmodel_v2 		 Params: 41941	 ROC-AUC: 97.17%, Accuracy: 91.62%, BRR: 300.47
topodnnmodel_v4 		 Params: 16013	 ROC-AUC: 97.20%, Accuracy: 91.63%, BRR: 304.55
topodnnmodel 		 Params: 59323	 ROC-AUC: 97.12%, Accuracy: 91.55%, BRR: 278.12
